Write SQL scripts that:  
a. modify the type of a column;  
b. add / remove a column;  
c. add / remove a DEFAULT constraint;  
d. add / remove a primary key;  
e. add / remove a candidate key;  
f. add / remove a foreign key;  
g. create / drop a table.

For each of the scripts above, write another one that reverts the operation. Place each script in a stored procedure. Use a simple, intuitive naming convention.

Create a new table that holds the current version of the database schema. Simplifying assumption: the version is an integer number.

Write a stored procedure that receives as a parameter a version number and brings the database to that version.

In [ ]:
use f1_season_2024;

a. modify the type of a column

In [ ]:
CREATE PROCEDURE ModifyTypeOfColumn AS
BEGIN
    ALTER TABLE TestTable ALTER COLUMN TestColumn INT;
END;

In [ ]:
CREATE PROCEDURE RevertModifyTypeOfColumn AS
BEGIN
    ALTER TABLE TestTable ALTER COLUMN TestColumn VARCHAR(255);
END;

b. add/ remove a column;

In [ ]:
CREATE PROCEDURE AddColumn AS
BEGIN
    ALTER TABLE TestTable ADD MyNewColumn NVARCHAR(50);
END;

In [ ]:
CREATE PROCEDURE RemoveColumn AS
BEGIN
    ALTER TABLE TestTable DROP COLUMN MyNewColumn;
END;

c. add / remove a DEFAULT constraint;

In [ ]:
CREATE PROCEDURE AddDefaultConstraint AS
BEGIN
    ALTER TABLE TestTable ADD CONSTRAINT DF_age DEFAULT 18 FOR age;
END;

In [ ]:
CREATE PROCEDURE RemoveDefaultConstraint AS
BEGIN
    ALTER TABLE TestTable DROP CONSTRAINT DF_age;
END;

d. add / remove a primary key

In [ ]:
CREATE PROCEDURE AddPrimaryKey
AS
BEGIN
    ALTER TABLE TestTable
    ADD CONSTRAINT PK_test_table PRIMARY KEY (id);
END;


In [ ]:
CREATE PROCEDURE RemovePrimaryKey
AS
BEGIN
    ALTER TABLE TestTable
    DROP CONSTRAINT PK_test_table;
END;

e. add / remove a candidate key (uniqueness constraint)

In [ ]:
CREATE PROCEDURE AddCandidateKey
AS
BEGIN
    ALTER TABLE TestTable
    ADD CONSTRAINT UQ_test_table UNIQUE (id);
END;

In [ ]:
CREATE PROCEDURE RemoveCandidateKey
AS
BEGIN
    ALTER TABLE TestTable
    DROP CONSTRAINT UQ_test_table;
END;

f. add / remove a foreign key

In [ ]:
CREATE PROCEDURE AddForeignKey
AS
BEGIN
    ALTER TABLE TestTable
    ADD CONSTRAINT FK_test_table FOREIGN KEY(TestColumn) REFERENCES driver(driverId);
END;

In [ ]:
CREATE PROCEDURE RemoveForeignKey
AS
BEGIN
    ALTER TABLE TestTable
    DROP CONSTRAINT FK_test_table
END;

g. create/drop table

In [ ]:
CREATE PROCEDURE CreateTable AS
BEGIN
    CREATE TABLE TestTable (
        id INT NOT NULL, -- Needs to be 'not null' in case we want the id to be the PK of the table
        name VARCHAR(255) NOT NULL,
        age INT NOT NULL,
        TestColumn INT NOT NULL
    );
END;

In [ ]:
CREATE PROCEDURE RemoveTable AS
BEGIN
    DROP TABLE TestTable;
END;

Now we need the table to keep the database version in

In [ ]:
CREATE TABLE db_version (
    versionId INT PRIMARY KEY,
    version INT NOT NULL
);

In [ ]:
INSERT INTO db_version (versionId, version) VALUES
    (0, 1); -- version 1
-- version 1 will not have the test table
-- version 2 will have the test table

In [ ]:
CREATE PROCEDURE ChangeVersion 
  @version INT
AS
BEGIN
    -- retrieve the current version
    DECLARE @currentVersion INT
    SELECT @currentVersion = version FROM db_version;

    IF @currentVersion = @version
    BEGIN
        -- nothing to do, print a message
        PRINT 'Database is already at version ' + CAST(@version AS VARCHAR);
        RETURN;
    END
    ELSE
    BEGIN
        IF @version = 1
        BEGIN
            -- upgrade to version 1
            PRINT 'Upgrading to version 1';
            -- do the upgrade
            UPDATE db_version SET version = 1;
            -- do some procedures
            EXEC RemoveTable;
            RETURN;
        END
        ELSE IF @version = 2
        BEGIN
            -- upgrade to version 2
            PRINT 'Upgrading to version 2';
            -- do the upgrade
            UPDATE db_version SET version = 2;
            -- do some procedures
            EXEC CreateTable;
            EXEC ModifyTypeOfColumn;
            EXEC AddColumn;
            EXEC AddDefaultConstraint;
            EXEC AddCandidateKey;
            EXEC AddPrimaryKey;
            EXEC AddForeignKey;
            RETURN;
        END
    END
END


In [ ]:
SELECT * FROM db_version;

In [ ]:
EXEC ChangeVersion @version = 2;

In [ ]:
select * from TestTable;